# Assignment 9

A RAID 1 in a very critical environment is composed by two disks that mirror the same data.
On average, every **MTTF = 100** days one of the disks fails.

In this case, recovery consist in replacing the broken disk, reconstruct the data, and return to normal operation.
This requires an average of **MTTR = 5** days.

Should also the second disk fail during the recovery stage, the system will go in a fault state, where to be restored, it will require a total reconstruction of the system from the backup.
This will take an average of **15** days.

## Requests
Considering a starting state where both disks are working:
- Draw the Markov Chain of the model
- Compute the infinitesimal generator and solve the corresponding differential equations
- Show the probability of the various states for the time T = [0, 10000] days

## Drawing of the Markov Chain of the model
In the drawing, the blue ellipses are the states of the chain and the edges are used to identify the rates of each exponential distribution.



The distributions' rates associated with each edge are:

- *MTTF* :  100
- *MTTR* :  5
- *Complete Repair* :  15

## Computation of the infinitesimal generator

The infinitesimal generator for the system is 

$$Q = \begin{bmatrix}
-\frac{2}{MTTF} & \frac{1}{MTTF} & \frac{1}{MTTF} & 0 \\
\frac{1}{MTTR} & -\frac{1}{MTTR}-\frac{1}{MTTF} & 0 & \frac{1}{MTTF} \\
\frac{1}{MTTR} & 0 & -\frac{1}{MTTR}-\frac{1}{MTTF} & \frac{1}{MTTF} \\
\frac{1}{complete\_repair} & 0 & 0 & -\frac{1}{complete\_repair}
\end{bmatrix}$$

In [1]:
import numpy as np

MTTF = 100
MTTR = 5
comp_rep = 15

Q = np.array([
    [   -2/MTTF,              1/MTTF,            1/MTTF,          0],
    [    1/MTTR,  -(1/MTTR)-(1/MTTF),                 0,     1/MTTF],
    [    1/MTTR,                   0,-(1/MTTR)-(1/MTTF),     1/MTTF],
    [1/comp_rep,                   0,                 0,-1/comp_rep]
])

## Computation of the probabilities

We can compute the probabilities in the interval requested by using a differential equation solver.

Since the starting state is considered to be both disk working, the initial probability of the chain will be:
$$\pi(0) =  \begin{bmatrix}
1 & 0 & 0 & 0
\end{bmatrix}$$

In [2]:
import scipy.integrate as spi

def diff_function(t, pi):
    return np.matmul(pi, Q)

p0 = [1,0,0,0]

Tmax = 10000

#The Radau method choice is empirical: the values I obtain with it are monotonically crescent.
#sol = spi.solve_ivp(diff_function, (0, Tmax), p0, method='Radau')
sol = spi.solve_ivp(diff_function, (0, Tmax), p0, t_eval = np.linspace(0,Tmax,100000))

#sol = spi.odeint(diff_function, p0, np.linspace(0,Tmax,100), tfirst=True)
# If I use the odeint function (similar to ode45 in matlab) I obtain a sudden angle in the curve

In [3]:
# for spi.solve_ivp
#for i in range(0,len(sol.y[0])):
#   print("{}\t{}\t{}\t{}".format(sol.y[0][i], sol.y[1][i], sol.y[2][i], sol.y[3][i]))

In [4]:
print("The probabilities for each state are:\n\n"
        "{!s:20}{:10%}\n"
        "{!s:20}{:10%}\n"
        "{!s:20}{:10%}\n"
        "{!s:20}{:10%}".format("Both working:",sol.y[0][-1],"first disk broke:", sol.y[1][-1],"second disk broke:", sol.y[2][-1],"Fault:", sol.y[3][-1]))

The probabilities for each state are:

Both working:       90.132139%
first disk broke:    4.290043%
second disk broke:   4.290043%
Fault:               1.287776%


The following is a time plot of the three probabilities

In [5]:
from matplotlib import pyplot as plt
%matplotlib tk
plt.plot(sol.t, sol.y[0], label="Both working")
plt.plot(sol.t, sol.y[1], label="first disk broke")
plt.plot(sol.t, sol.y[2], label="second disk broke")
plt.plot(sol.t, sol.y[3], label="Fault")
plt.legend()

### Steady state probabilities
We can also compute the steady state probabilities of the chain

In [6]:
Q_steady = Q.copy()
Q_steady[:,0] = [1, 1, 1, 1]

ss_probabilities = np.matmul([1, 0, 0, 0], np.linalg.inv(Q_steady))

print("The probabilities for each state are:\n\n"
        "{!s:20}{:10%}\n"
        "{!s:20}{:10%}\n"
        "{!s:20}{:10%}\n"
        "{!s:20}{:10%}".format("Both working:",ss_probabilities[0],"first disk broke:", ss_probabilities[1],"second disk broke:", ss_probabilities[2],"Fault:", ss_probabilities[3]))

The probabilities for each state are:

Both working:       90.128755%
first disk broke:    4.291845%
second disk broke:   4.291845%
Fault:               1.287554%
